## Import Libraries

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Read Config Files

In [ ]:
import configparser 
config = configparser.ConfigParser()
config.read('config.ini')
ip = config['DEFAULT']['IP']
port = config['DEFAULT']['MongoDB-Port']
db_name = config['DEFAULT']['DB-Name']
collection_name = config['DEFAULT']['Collection-Name']

## Connect MongoDB

In [ ]:
from pymongo import MongoClient
client = MongoClient(ip, int(port))

In [ ]:
db = client[db_name]

## Pipeline

In [ ]:
# For language field
pipeline_lang = [
    {"$match": {"lang" : "en"}},
    {"$count": "number_english"}
]

In [ ]:
# For geoname field
pipeline_geoname = [
    {"$match": {"geoname": {"$exists":True}}},
    {"$count": "number_geoname"}
]

## Aggregation

In [ ]:
lang = list(db[collection_name].aggregate(pipeline_lang,allowDiskUse=True))
print(lang)


In [ ]:
geoname = list(db[collection_name].aggregate(pipeline_geoname,allowDiskUse=True))
print(geoname)
